In [1]:
import qiskit
from qiskit.circuit import QuantumCircuit

为了实现面向对象，我们这里需要创建一个Circuit类型，类型里面需要存储指令，指令执行顺序，最好利用可迭代对象实现。
指令也是一个类型，其中需要存储指令的名称与指令作用的比特位置。

In [5]:
from typing import List, Iterator

class Instruction:
    def __init__(self, name: str, qubits: List[int]):
        self.name = name
        self.qubits = qubits

    def __repr__(self):
        return f"Instruction(name={self.name}, qubits={self.qubits})"

class Circuit:
    def __init__(self, regs: int):
        self.qubits = regs
        self.instructions = []

    def add_instruction(self, instruction: Instruction):
        self.instructions.append(instruction)

    def __iter__(self) -> Iterator[Instruction]:
        return iter(self.instructions)

    def __repr__(self):
        return f"Circuit(instructions={self.instructions})"

# 示例使用
instr1 = Instruction(name="H", qubits=[0])
instr2 = Instruction(name="CNOT", qubits=[0, 1])
instr3 = Instruction(name="X", qubits=[1])

circuit = Circuit(2)
circuit.add_instruction(instr1)
circuit.add_instruction(instr2)
circuit.add_instruction(instr3)

# 迭代电路中的指令
for instr in circuit:
    print(instr)

Instruction(name=H, qubits=[0])
Instruction(name=CNOT, qubits=[0, 1])
Instruction(name=X, qubits=[1])


创建Bell电路

In [10]:
H_0 = Instruction(name="h", qubits=[0])
CNOT_01 = Instruction(name="cnot", qubits=[0, 1])
Measure_01 = Instruction(name="measure", qubits=[0, 1])

Bell_circuit = Circuit(2)
Bell_circuit.add_instruction(H_0)
Bell_circuit.add_instruction(CNOT_01)
Bell_circuit.add_instruction(Measure_01)

for instr in Bell_circuit:
    print(instr)

Instruction(name=h, qubits=[0])
Instruction(name=cnot, qubits=[0, 1])
Instruction(name=measure, qubits=[0, 1])


实现电路的模拟。

In [30]:
import numpy as np

class Instruction:
    def __init__(self, name: str, qubits: List[int]):
        self.name = name
        self.qubits = qubits

    def __repr__(self):
        return f"Instruction(name={self.name}, qubits={self.qubits})"

class Circuit:
    def __init__(self, num_qubits: int):
        self.num_qubits = num_qubits
        self.instructions = []

    def add_instruction(self, instruction: Instruction):
        self.instructions.append(instruction)

    def simulate(self):
        # 初始量子态为 |0...0>
        state = np.zeros(2**self.num_qubits, dtype=complex)
        state[0] = 1.0  # |0...0>

        for instr in self.instructions:
            if instr.name == "H":
                state = self.apply_single_qubit_gate(state, self.hadamard_gate(), instr.qubits[0])
            elif instr.name == "X":
                state = self.apply_single_qubit_gate(state, self.x_gate(), instr.qubits[0])
            elif instr.name == "CNOT":
                state = self.apply_cnot(state, instr.qubits[0], instr.qubits[1])
        
        return state

    def hadamard_gate(self):
        return (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])

    def x_gate(self):
        return np.array([[0, 1], [1, 0]])

    def apply_single_qubit_gate(self, state, gate, qubit):
        full_gate = 1
        for i in range(self.num_qubits):
            if i == qubit:
                full_gate = np.kron(full_gate, gate)
            else:
                full_gate = np.kron(full_gate, np.eye(2))
        return full_gate @ state

    def apply_cnot(self, state, control, target):
        dim = 2 ** self.num_qubits
        CNOT = np.eye(dim, dtype=complex)
        for i in range(dim):
            binary = format(i, f'0{self.num_qubits}b')
            if binary[control] == '1':
                flipped = binary[:target] + ('0' if binary[target] == '1' else '1') + binary[target+1:]
                j = int(flipped, 2)
                CNOT[i, i] = 0
                CNOT[i, j] = 1
        return CNOT @ state

# 示例使用
instr1 = Instruction(name="H", qubits=[0])
instr2 = Instruction(name="CNOT", qubits=[0, 1])
# instr3 = Instruction(name="X", qubits=[1])
# instr4 = Instruction(name="H", qubits=[2])
# instr5 = Instruction(name="CNOT", qubits=[1, 2])

circuit = Circuit(num_qubits=3)
circuit.add_instruction(instr1)
circuit.add_instruction(instr2)
# circuit.add_instruction(instr3)
# circuit.add_instruction(instr4)
# circuit.add_instruction(instr5)

# 模拟电路
final_state = circuit.simulate()
print("Final quantum state:", final_state)


Final quantum state: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]


CP门和SWAP门。

In [36]:
import numpy as np

class Instruction:
    def __init__(self, name: str, qubits: List[int], parameter: float = None):
        self.name = name
        self.qubits = qubits
        self.parameter = parameter  # 用于CP门的相位参数

    def __repr__(self):
        return f"Instruction(name={self.name}, qubits={self.qubits}, parameter={self.parameter})"

class Circuit:
    def __init__(self, num_qubits: int):
        self.num_qubits = num_qubits
        self.instructions = []

    def add_instruction(self, instruction: Instruction):
        self.instructions.append(instruction)

    def simulate(self):
        # 初始量子态为 |0...0>
        state = np.zeros(2**self.num_qubits, dtype=complex)
        state[0] = 1.0  # |0...0>

        for instr in self.instructions:
            if instr.name == "H":
                state = self.apply_single_qubit_gate(state, self.hadamard_gate(), instr.qubits[0])
            elif instr.name == "X":
                state = self.apply_single_qubit_gate(state, self.x_gate(), instr.qubits[0])
            elif instr.name == "CNOT":
                state = self.apply_cnot(state, instr.qubits[0], instr.qubits[1])
            elif instr.name == "CP":
                state = self.apply_cp(state, instr.qubits[0], instr.qubits[1], instr.parameter)
            elif instr.name == "SWAP":
                state = self.apply_swap(state, instr.qubits[0], instr.qubits[1])
        
        return state

    def hadamard_gate(self):
        return (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])

    def x_gate(self):
        return np.array([[0, 1], [1, 0]])

    def apply_single_qubit_gate(self, state, gate, qubit):
        full_gate = 1
        for i in range(self.num_qubits):
            if i == qubit:
                full_gate = np.kron(full_gate, gate)
            else:
                full_gate = np.kron(full_gate, np.eye(2))
        return full_gate @ state

    def apply_cnot(self, state, control, target):
        dim = 2 ** self.num_qubits
        CNOT = np.eye(dim, dtype=complex)
        
        for i in range(dim):
            binary = format(i, f'0{self.num_qubits}b')
            if binary[control] == '1':
                flipped = binary[:target] + ('0' if binary[target] == '1' else '1') + binary[target+1:]
                j = int(flipped, 2)
                CNOT[i, i] = 0
                CNOT[i, j] = 1

        return CNOT @ state

    def apply_cp(self, state, control, target, theta):
        dim = 2 ** self.num_qubits
        CP = np.eye(dim, dtype=complex)

        for i in range(dim):
            binary = format(i, f'0{self.num_qubits}b')
            if binary[control] == '1' and binary[target] == '1':
                CP[i, i] = np.exp(1j * theta)

        return CP @ state

    def apply_swap(self, state, qubit1, qubit2):
        dim = 2 ** self.num_qubits
        SWAP = np.eye(dim, dtype=complex)

        for i in range(dim):
            binary = format(i, f'0{self.num_qubits}b')
            swapped = list(binary)
            swapped[qubit1], swapped[qubit2] = swapped[qubit2], swapped[qubit1]
            j = int(''.join(swapped), 2)
            SWAP[i, i] = 0
            SWAP[i, j] = 1

        return SWAP @ state
    
    def __iter__(self) -> Iterator[Instruction]:
        return iter(self.instructions)

    def __repr__(self):
        return f"Circuit(instructions={self.instructions})"

# 示例使用
instr1 = Instruction(name="H", qubits=[0])
instr2 = Instruction(name="CNOT", qubits=[0, 1])
instr3 = Instruction(name="X", qubits=[1])
instr4 = Instruction(name="H", qubits=[2])
instr5 = Instruction(name="CNOT", qubits=[1, 2])
instr6 = Instruction(name="CP", qubits=[0, 2], parameter=np.pi/4)
instr7 = Instruction(name="SWAP", qubits=[0, 2])

circuit = Circuit(num_qubits=3)
circuit.add_instruction(instr1)
circuit.add_instruction(instr2)
circuit.add_instruction(instr3)
circuit.add_instruction(instr4)
circuit.add_instruction(instr5)
circuit.add_instruction(instr6)
circuit.add_instruction(instr7)

# 模拟电路
final_state = circuit.simulate()
print("Final quantum state:", final_state)


Final quantum state: [0.        +0.j         0.5       +0.j         0.5       +0.j
 0.        +0.j         0.        +0.j         0.35355339+0.35355339j
 0.5       +0.j         0.        +0.j        ]


In [40]:
# 构建5比特QFT电路
def build_qft_circuit(num_qubits):
    circuit = Circuit(num_qubits)

    # 应用Hadamard门
    for qubit in range(num_qubits):
        circuit.add_instruction(Instruction(name="H", qubits=[qubit]))

    # 应用控制相位门
    for control in range(num_qubits):
        circuit.add_instruction(Instruction(name="H", qubits=[control]))
        for target in range(control + 1, num_qubits):
            theta = 2 * np.pi / (2 ** (target - control))
            circuit.add_instruction(Instruction(name="CP", qubits=[control, target], parameter=theta))

    # 反序SWAP门
    for qubit in range(num_qubits // 2):
        circuit.add_instruction(Instruction(name="SWAP", qubits=[qubit, num_qubits - qubit - 1]))

    return circuit

# 示例使用
num_qubits = 5
qft_circuit = build_qft_circuit(num_qubits)

for instr in qft_circuit:
    print(instr)
    
final_state = qft_circuit.simulate()

# 输出最终量子态
print("Final quantum state (QFT):")
print(final_state)

Instruction(name=H, qubits=[0], parameter=None)
Instruction(name=H, qubits=[1], parameter=None)
Instruction(name=H, qubits=[2], parameter=None)
Instruction(name=H, qubits=[3], parameter=None)
Instruction(name=H, qubits=[4], parameter=None)
Instruction(name=H, qubits=[0], parameter=None)
Instruction(name=CP, qubits=[0, 1], parameter=3.141592653589793)
Instruction(name=CP, qubits=[0, 2], parameter=1.5707963267948966)
Instruction(name=CP, qubits=[0, 3], parameter=0.7853981633974483)
Instruction(name=CP, qubits=[0, 4], parameter=0.39269908169872414)
Instruction(name=H, qubits=[1], parameter=None)
Instruction(name=CP, qubits=[1, 2], parameter=3.141592653589793)
Instruction(name=CP, qubits=[1, 3], parameter=1.5707963267948966)
Instruction(name=CP, qubits=[1, 4], parameter=0.7853981633974483)
Instruction(name=H, qubits=[2], parameter=None)
Instruction(name=CP, qubits=[2, 3], parameter=3.141592653589793)
Instruction(name=CP, qubits=[2, 4], parameter=1.5707963267948966)
Instruction(name=H, qubi

增加对于量子态的测量操作。

In [57]:
import numpy as np
from numpy.random import default_rng

class Instruction:
    def __init__(self, name: str, qubits: List[int], parameter: float = None):
        self.name = name
        self.qubits = qubits
        self.parameter = parameter  # 用于CP门的相位参数

    def __repr__(self):
        return f"Instruction(name={self.name}, qubits={self.qubits}, parameter={self.parameter})"

class Circuit:
    def __init__(self, num_qubits: int):
        self.num_qubits = num_qubits
        self.instructions = []

    def add_instruction(self, instruction: Instruction):
        self.instructions.append(instruction)

    def simulate(self):
        # 初始量子态为 |0...0>
        state = np.zeros(2**self.num_qubits, dtype=complex)
        state[0] = 1.0  # |0...0>

        for instr in self.instructions:
            if instr.name == "H":
                state = self.apply_single_qubit_gate(state, self.hadamard_gate(), instr.qubits[0])
            elif instr.name == "X":
                state = self.apply_single_qubit_gate(state, self.x_gate(), instr.qubits[0])
            elif instr.name == "CNOT":
                state = self.apply_cnot(state, instr.qubits[0], instr.qubits[1])
            elif instr.name == "CP":
                state = self.apply_cp(state, instr.qubits[0], instr.qubits[1], instr.parameter)
            elif instr.name == "SWAP":
                state = self.apply_swap(state, instr.qubits[0], instr.qubits[1])
        
        # # 测量最终量子态
        # if measure:
        #     measured_state = self.measure_state(state)
        #     return measured_state
        # else:
        return state

    def hadamard_gate(self):
        return (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])

    def x_gate(self):
        return np.array([[0, 1], [1, 0]])

    def apply_single_qubit_gate(self, state, gate, qubit):
        full_gate = 1
        for i in range(self.num_qubits):
            if i == qubit:
                full_gate = np.kron(full_gate, gate)
            else:
                full_gate = np.kron(full_gate, np.eye(2))
        return full_gate @ state

    def apply_cnot(self, state, control, target):
        dim = 2 ** self.num_qubits
        CNOT = np.eye(dim, dtype=complex)
        
        for i in range(dim):
            binary = format(i, f'0{self.num_qubits}b')
            if binary[control] == '1':
                flipped = binary[:target] + ('0' if binary[target] == '1' else '1') + binary[target+1:]
                j = int(flipped, 2)
                CNOT[i, i] = 0
                CNOT[i, j] = 1

        return CNOT @ state

    def apply_cp(self, state, control, target, theta):
        dim = 2 ** self.num_qubits
        CP = np.eye(dim, dtype=complex)

        for i in range(dim):
            binary = format(i, f'0{self.num_qubits}b')
            if binary[control] == '1' and binary[target] == '1':
                CP[i, i] = np.exp(1j * theta)

        return CP @ state

    def apply_swap(self, state, qubit1, qubit2):
        dim = 2 ** self.num_qubits
        SWAP = np.eye(dim, dtype=complex)

        for i in range(dim):
            binary = format(i, f'0{self.num_qubits}b')
            swapped = list(binary)
            swapped[qubit1], swapped[qubit2] = swapped[qubit2], swapped[qubit1]
            j = int(''.join(swapped), 2)
            SWAP[i, i] = 0
            SWAP[i, j] = 1

        return SWAP @ state
    
    def __iter__(self) -> Iterator[Instruction]:
        return iter(self.instructions)

    def __repr__(self):
        return f"Circuit(instructions={self.instructions})"

def measure_state(state, num_qubits):
    # 计算测量概率
    probabilities = np.abs(state)**2
    # 生成随机数
    rng = default_rng()
    rand_num = rng.random()
    # 执行测量
    cumulative_prob = 0.0
    for idx, prob in enumerate(probabilities):
        cumulative_prob += prob
        if rand_num < cumulative_prob:
            # measured_state = np.zeros_like(state)
            # measured_state[idx] = 1.0
            measured_state = format(idx, f'0{num_qubits}b')
            return measured_state
            # return measured_state


# 示例使用
instr1 = Instruction(name="H", qubits=[0])
# instr2 = Instruction(name="CNOT", qubits=[0, 1])
# instr3 = Instruction(name="X", qubits=[1])
# instr4 = Instruction(name="H", qubits=[2])
# instr5 = Instruction(name="CNOT", qubits=[1, 2])
# instr6 = Instruction(name="CP", qubits=[0, 2], parameter=np.pi/4)
# instr7 = Instruction(name="SWAP", qubits=[0, 2])

circuit = Circuit(num_qubits=3)
circuit.add_instruction(instr1)
# circuit.add_instruction(instr2)
# circuit.add_instruction(instr3)
# circuit.add_instruction(instr4)
# circuit.add_instruction(instr5)
# circuit.add_instruction(instr6)
# circuit.add_instruction(instr7)

# 模拟电路
final_state = circuit.simulate()
counts = {}
for i in range(1024):
    m = measure_state(final_state,3)
    counts[m] = counts.get(m,0) + 1

print("Final quantum state:", final_state)
print("Result counts", counts)

Final quantum state: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
Result counts {'000': 510, '100': 514}


In [114]:
import numpy as np

# 创建一个10维量子态
qubits = 4
dimension = 2**qubits  # 10个量子比特
quantum_state = np.zeros(dimension)
quantum_state[0] = np.sqrt(1/2)  # 假设初始态为|000⟩ + |011⟩
quantum_state[6] = np.sqrt(1/2)

# 选择要测量的5个量子比特
qubits_to_measure = [1,2]  
qubits_not_to_measure = [i for i in range(qubits) if i not in qubits_to_measure]
num_qubits_to_measure = len(qubits_to_measure)


# 生成所有可能的n位二进制字符串
all_binaries = np.array(['{:04b}'.format(i) for i in range(dimension)])



# 生成测量矩阵
measurement_matrix = np.zeros((2**num_qubits_to_measure, dimension))

matrix = np.array([2**i for i in range(qubits)])
for i in range(2**num_qubits_to_measure):
    
    basis_state = np.binary_repr(i, width=num_qubits_to_measure)[::-1]
    # 找出第二位和第三位为0的字符串
    for b in all_binaries:

        if 
    filtered_binaries = np.array([b for b in all_binaries if b[0] == '1' and b[2] == '1'])
    # basis_index = sum([int(b) * 2**j for j, b in enumerate(basis_state)])
    print(basis_state)
    # measurement_matrix[i, basis_index] = 1.0
    print(i)
    # basis_index = 0
    # for j, qubit in enumerate(qubits_to_measure):
    #     basis_index += int(basis_state[j]) * 2**qubit
    #     # measurement_matrix[i, basis_index] = 1.0
    #     # 其他全部为0
    # for not_measure_i in range(2**(qubits-num_qubits_to_measure)):
    #     not_measure_state = np.binary_repr(not_measure_i, width=qubits-num_qubits_to_measure)[::-1]
    #     print("not_measure_state",not_measure_state)
    #     for j, qubit in enumerate(qubits_to_measure):
    #     basis_index += int(basis_state[j]) * 2**qubit
        
    # print("basis_index", basis_index)
    # measurement_matrix[i, basis_index] = 1.0

# 进行测量
# measured_state = measurement_matrix @ quantum_state

# 打印测量结果
# print("Measured State:")
# print(measured_state)

00
0
10
1
01
2
11
3


array(['0001', '0010'], dtype='<U4')

In [119]:
# 生成所有可能的4位二进制字符串
all_binaries = np.array(['{:04b}'.format(i)[::-1] for i in range(16)])



In [120]:
all_binaries

array(['0000', '1000', '0100', '1100', '0010', '1010', '0110', '1110',
       '0001', '1001', '0101', '1101', '0011', '1011', '0111', '1111'],
      dtype='<U4')

In [125]:
# 找出第二位和第三位为0的字符串
filtered_binaries = all_binaries[(all_binaries[:][1] == '0') & (all_binaries[:][2] == '0')]

In [126]:
filtered_binaries

array([], shape=(0, 16), dtype='<U4')

In [130]:
import numpy as np

# 生成所有可能的4位二进制字符串
all_binaries = np.array(['{:04b}'.format(i) for i in range(16)])

# 找出第二位和第三位为0的字符串
filtered_binaries = np.array([b for b in all_binaries if b[0] == '1' and b[2] == '1'])

print(filtered_binaries)

['1010' '1011' '1110' '1111']


In [83]:
sum([int(b) * 2**(j) for j, b in enumerate("10")])

1

In [93]:
np.binary_repr(-1, width=2)

'11'